# Liver Model

## Import some things

In [8]:
from Liver_Generator import Data_Generator, os, plot_scroll_Image, dice_coef_3D
from Utils import ModelCheckpoint, model_path_maker
from Callbacks.Visualizing_Model_Utils import TensorBoardImage
import tensorflow as tf
import keras.backend as K

c:\users\bmanderson\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\bmanderson\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [ ]:
%matplotlib inline

In [4]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [5]:
data_path = os.path.join('..','Data','Niftii_Arrays')
train_path = os.path.join(data_path,'Train')
validation_path = os.path.join(data_path,'Validation')
test_path = os.path.join(data_path,'Test')
model_path = os.path.join('..','Models')
if not os.path.exists(model_path):
    os.makedirs(model_path)

In [ ]:
args = {'batch_size':5,'on_vgg':True,'mean_val':78,'std_val':29}#'mean_val':78,'std_val':29
train_generator = Data_Generator(train_path, shuffle=True, **args)

## Normalize images about the mask to be mean 0, std 1

In [ ]:
#train_generator.get_mean_std_val() # This will calculate the mean and std for you and set it in the model
print(train_generator.mean_val)

In [ ]:
args_val = {'on_vgg':True,'mean_val':train_generator.mean_val,'std_val':train_generator.std_val,'by_patient':False,
       'shuffle':True,'batch_size':20}
validation_generator = Data_Generator(validation_path, **args_val)
args_val = {'on_vgg':True,'mean_val':train_generator.mean_val,'std_val':train_generator.std_val,'by_patient':True,
       'shuffle':False}
test_generator = Data_Generator(test_path, **args_val)

In [ ]:
x,y = train_generator.__getitem__(0)

In [ ]:
plot_scroll_Image(x[...,-1])

### Alright, lets make our model!

In [ ]:
from Easy_VGG16_UNet.Keras_Fine_Tune_VGG_16_Liver import VGG_16
from Visualizing_Model.Visualing_Model import visualization_model_class
from keras.optimizers import Adam
import tensorflow as tf
import keras.backend as K

In [ ]:
K.clear_session()
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
K.set_session(sess)
network = {'Layer_0': {'Encoding': [64, 64], 'Decoding': [32]},
           'Layer_1': {'Encoding': [128, 128], 'Decoding': [64]},
           'Layer_2': {'Encoding': [256, 256, 256], 'Decoding': [256]},
           'Layer_3': {'Encoding': [512, 512, 512], 'Decoding': [256]},
           'Layer_4': {'Encoding': [512, 512, 512]}}
VGG_model = VGG_16(network=network, activation='relu',filter_size=(3,3))
VGG_model.make_model()
VGG_model.load_weights()
new_model = VGG_model.created_model
model_path = os.path.join('..','Models')

## These are some tools for visualizing the model

In [ ]:
Visualizing_Class = visualization_model_class(model=new_model, save_images=True, verbose=True)

In [ ]:
Visualizing_Class.define_desired_layers(['block1_conv1','block1_conv1_activation','Output'])

In [ ]:
Visualizing_Class.predict_on_tensor(x[0,...][None,...])

In [ ]:
Visualizing_Class.plot_activations()

In [ ]:
new_model.compile(Adam(lr=5e-5),loss='categorical_crossentropy', metrics=['accuracy',dice_coef_3D])

## Freezing pre-trained layers

In [ ]:
def freeze_until_name(model,name):
    set_trainable = False
    for layer in model.layers:
        if layer.name == name:
            set_trainable = True
        layer.trainable = set_trainable
    return model
new_model = freeze_until_name(new_model,'Upsampling0_UNet')

## Checkpoint and run

A checkpoint is a way of assessing the model and determining if we should save it

In [13]:
model_name = 'VGG_16_Model'
other_aspects = [model_name,'Upsampling0_UNet_Unfrozen'] # Just a list of defining things
model_path_out = model_path_maker(model_path,other_aspects)

NameError: name 'model_path' is not defined

In [ ]:
checkpoint = ModelCheckpoint(os.path.join(model_path_out,'best-model.hdf5'), monitor='val_dice_coef_3D', verbose=1, save_best_only=True,
                              save_weights_only=False, period=5, mode='max')
# TensorboardImage lets us view the predictions of our model
tensorboard = TensorBoardImage(log_dir=model_path_out, batch_size=1, num_images=3,update_freq='epoch', 
                               data_generator=validation_generator)
callbacks = [checkpoint, tensorboard]

In [10]:
%tensorboard --logdir {"../Models"}

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 18088.

In [ ]:
#new_model.fit_generator(train_generator,epochs=1, workers=20, max_queue_size=50, validation_data=validation_generator,callbacks=callbacks)

In [ ]:
x,y = validation_generator.__getitem__(0)

In [ ]:
pred = new_model.predict(x)

In [ ]:
pred[pred<0.5] = 0
pred[pred>0] = 1

In [ ]:
plot_scroll_Image(pred[...,1])

# Now lets make our own architecture

In [ ]:
from UNet_Maker import my_UNet
from functools import partial

In [17]:
filters_start = 16
layers = 3
conv_blocks = 2
learning_rate = 1e-5

In [10]:
layers_dict = {}
filters = filters_start
pool = (4,4)
conv_block = lambda x: {'Channels': [x], 'Kernel': [(3, 3)]}
for layer in range(layers-1):
    layers_dict['Layer_{}'.format(layer)] = {'Encoding':[conv_block(filters) for _ in range(conv_blocks)],
                          'Pooling':pool,
                          'Decoding':[conv_block(filters) for _ in range(conv_blocks)]}
    filters *= 2
    pool = (2,2)
layers_dict['Base'] = {'Encoding':[conv_block(filters) for _ in range(conv_blocks)]}

In [ ]:
K.clear_session()
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
K.set_session(sess)
new_model = my_UNet(layers_dict=layers_dict,image_size=512,num_channels=train_generator.channels).created_model
new_model.compile(Adam(lr=learning_rate),loss='categorical_crossentropy', metrics=['accuracy',dice_coef_3D])

In [18]:
model_name = 'My_New_Model'
other_aspects = [model_name,'{}_Layers'.format(layers),'{}_Conv_Blocks'.format(conv_blocks),
                 '{}_Filters'.format(filters_start),'{}_Learning_Rate'.format(learning_rate)] # Just a list of defining things
model_path_out = model_path_maker(model_path,other_aspects)

In [ ]:
checkpoint = ModelCheckpoint(os.path.join(model_path_out,'best-model.hdf5'), monitor='val_dice_coef_3D', verbose=1, save_best_only=True,
                              save_weights_only=False, period=5, mode='max')
# TensorboardImage lets us view the predictions of our model
tensorboard = TensorBoardImage(log_dir=model_path_out, batch_size=1, num_images=3,update_freq='epoch', 
                               data_generator=validation_generator)
callbacks = [checkpoint, tensorboard]

In [ ]:
new_model.fit_generator(train_generator,epochs=5, workers=20, max_queue_size=50, validation_data=validation_generator,
                       callbacks=callbacks, steps_per_epoch=10)